Kalibracja kamery to proces określenia jej parametrów wewnętrznych i zewnętrznych, które pozwalają na dokładne odwzorowanie trójwymiarowych punktów sceny na dwuwymiarową płaszczyznę obrazu. Jets to kluczowych etap podczas pracy z danymi z czujników, gdyż pozwala skorygować zniekształcenia optyczne a także określić relacje przestrzenne kamery i układu odniesienia świata. 

Estymowane parametry kamery można podzielić na dwie grupy:
- parametry wewnętrzne (intrinsic parameters) K - opisują właściwości optyki i matrycy kamery
    - ogniskowa fx i fy
    - punktu główny cx i cy
    - współczynniki dystorsji optycznej: k1,k2,k3, p1, p2
- parametry zewnętrzne (extrinsic parameters) - określają pozycje i orientację kamery w przestrzenii
    - macierz rotacji R
    - wektor translacji T
